In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [22]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using AdaptiveTransportMap: derivative, vander, transform!, evaluate, ncoeff, optimize
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using Polynomials
using Optim: InverseDiagonal
using MLDataUtils

### Greedy fit

In [23]:
Nx = 2

m = 20
Ne = 200
X = randn(Nx, Ne).^2 + 0.1*randn(Nx, Ne)

L = LinearTransform(X; diag = true)
transform!(L, X);

In [24]:
Hk = HermiteMapk(m, Nx);

In [33]:
AdaptiveTransportMap.optimize(Hk, X, "split");